# Tests de funcionalidad de la API de series de tiempo

In [5]:
import os
import pandas as pd
import numpy as np
import requests
from io import StringIO

## Variables

**Obligatorio**: setear la variable de entorno API_URL o setear la variable BASE_URL al ambiente de la API que se quiere probar

In [14]:
BASE_URL = "http://186.33.215.34/"
METADATA_URL = 'http://infra.datos.gob.ar/catalog/modernizacion/dataset/1/distribution/1.2/download/series-tiempo-metadatos.csv'
ENDPOINT_URL = BASE_URL + 'series/api/series/'

In [15]:
series_metadata = pd.read_csv(METADATA_URL)
series_metadata.tail(5)

,catalogo_id,dataset_id,distribucion_id,serie_id,indice_tiempo_frecuencia,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,...,dataset_descripcion,dataset_tema,serie_indice_inicio,serie_indice_final,serie_valores_cant,serie_dias_no_cubiertos,serie_actualizada,serie_valor_ultimo,serie_valor_anterior,serie_var_pct_anterior
20190,sspm,99,99.3,99.3_ING_2008_0_17,R/P1M,ipc_nivel_general,Índice abr-2008=100,Índice de precios al consumidor nivel general....,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1633,False,166.840000,164.510000,0.014163
20191,sspm,99,99.3,99.3_IR_2008_0_13,R/P1M,ipc_regulados,Índice abr-2008=100,Índice de precios al consumidor regulados. Val...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1633,False,161.040000,159.290000,0.010986
20192,sspm,99,99.3,99.3_IR_2008_0_9,R/P1M,ipc_resto,Índice abr-2008=100,Índice de precios al consumidor IPC resto. Val...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1633,False,161.740000,159.250000,0.015636
20193,sspm,99,99.3,99.3_PREIR_2008_0_40,R/P1M,precios_relativos_estacionales_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1633,False,137.591196,138.411303,-0.005925
20194,sspm,99,99.3,99.3_PRRIR_2008_0_37,R/P1M,precios_relativos_regulados_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1633,False,99.567207,100.025118,-0.004578


In [16]:
series_metadata.serie_id.count()

20195

## Chequeo de todas las series 

Le pegamos al endpoint para todas las series y verificamos que la API devuelve una respuesta satisfactoria (status code 200). Contamos la cantidad de casos satisfactorios (True) y no (False)

In [17]:
def api_series_head(serie_id):
    return requests.head(ENDPOINT_URL, params={'ids': serie_id}).status_code == 200

### Cantidad de respuestas válidas

In [19]:
valid_responses = series_metadata.serie_id[:10].apply(api_series_head)
valid_responses.value_counts()

True    10
Name: serie_id, dtype: int64

### En porcentajes

In [20]:
valid_responses.value_counts().apply(lambda x: x/len(valid_responses))

True    1.0
Name: serie_id, dtype: float64

## Chequeo de modos de representación

In [21]:
def api_call(serie, limit=1000, **kwargs):
    call_params = {'ids': serie, 'format': 'csv', 'limit': limit}
    call_params.update(kwargs)
    csv = StringIO(requests.get(ENDPOINT_URL, params=call_params).content.decode('utf8'))
    api_csv = pd.read_csv(csv, parse_dates=['indice_tiempo'], index_col='indice_tiempo')

    return api_csv

def get_source_csv(serie):
    response = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'only'}).json()
    distribution_url = response['meta'][1]['distribution']['downloadURL']
    title = response['meta'][1]['field']['title']
    orig_csv = pd.read_csv(distribution_url, parse_dates=['indice_tiempo'], index_col='indice_tiempo')

    return orig_csv, title

In [24]:
serie = series_metadata.serie_id.sample(1)

serie_idx = serie.index[0]
serie = serie.values[0]
serie

'125.1_PAA_1993_0_23'

### Serie original

In [25]:
orig_csv, title = get_source_csv(serie)

### Serie de la API: valor original

In [26]:
api_csv = api_call(serie)

api_csv.head(1)

,piedras_arenas_arcillas
indice_tiempo,
1993-01-01,100.000017


In [27]:
orig_csv = orig_csv[:len(api_csv)]
equality_check = np.isclose(orig_csv[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    22
dtype: int64

### Cambio absoluto

In [28]:
api_csv = api_call(serie, representation_mode='change')

In [29]:
orig_csv_change, title = get_source_csv(serie)
orig_csv_change[title] = orig_csv_change[title].diff(1)
orig_csv_change = orig_csv_change[:len(api_csv)]

In [30]:
equality_check = np.isclose(orig_csv_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    22
dtype: int64

### Cambio porcentual

In [31]:
api_csv = api_call(serie, representation_mode='percent_change')

In [32]:
orig_csv_pct_change, title = get_source_csv(serie)
orig_csv_pct_change[title] = orig_csv_pct_change[title].pct_change()
orig_csv_pct_change = orig_csv_pct_change[:len(api_csv)]

In [33]:
equality_check = np.isclose(orig_csv_pct_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    22
dtype: int64

### Colapsos de datos

Aplico las agregaciones máximo y mínimos de la API, y también al CSV original con pandas. Comparo los resultados

In [34]:
api_max = api_call(serie, collapse='year', collapse_aggregation='max')
api_max['api_max'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(max).join(api_max)

,piedras_arenas_arcillas,api_max
indice_tiempo,,
1993-01-01,100.000017,100.000017
1994-01-01,104.275875,104.275875
1995-01-01,102.986267,102.986267
1996-01-01,98.631408,98.631408
1997-01-01,99.960408,99.960408
1998-01-01,101.308650,101.308650
1999-01-01,99.722808,99.722808
2000-01-01,97.701800,97.701800
2001-01-01,97.312025,97.312025


In [35]:
api_max = api_call(serie, collapse='year', collapse_aggregation='min')
api_max['api_min'] = api_max[title]
del api_max[title]
api_call(serie).resample('AS').apply(min).join(api_max)

,piedras_arenas_arcillas,api_min
indice_tiempo,,
1993-01-01,100.000017,100.000017
1994-01-01,104.275875,104.275875
1995-01-01,102.986267,102.986267
1996-01-01,98.631408,98.631408
1997-01-01,99.960408,99.960408
1998-01-01,101.308650,101.308650
1999-01-01,99.722808,99.722808
2000-01-01,97.701800,97.701800
2001-01-01,97.312025,97.312025


### Metadata

In [36]:
orig_meta = series_metadata.iloc[serie_idx]
orig_meta

catalogo_id                                                               sspm
dataset_id                                                                 125
distribucion_id                                                          125.1
serie_id                                                   125.1_PAA_1993_0_23
indice_tiempo_frecuencia                                                 R/P1Y
serie_titulo                                           piedras_arenas_arcillas
serie_unidades                                                 Índice 1993=100
serie_descripcion                      Piedras, arenas y arcillas índice anual
distribucion_titulo          Índice de Precios Internos Básicos al por Mayo...
distribucion_descripcion     Índice de Precios Internos Básicos al por Mayo...
distribucion_url_descarga    http://infra.datos.gob.ar/catalog/sspm/dataset...
dataset_responsable              Subsecretaría de Programación Macroeconómica.
dataset_fuente               Instituto Nacional de E

In [37]:
metadata = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'full'}).json()

In [38]:
# Metadatos del índice de tiempo
metadata['meta'][0]

{'frequency': 'year', 'start_date': '1993-01-01', 'end_date': '2014-01-01'}

In [39]:
serie_meta = metadata['meta'][1]
serie_meta

{'catalog': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'license': 'Creative Commons Attribution 4.0',
  'description': 'Catálogo de datos abiertos de la Subsecretaría de Programación Macroeconómica.',
  'language': ['SPA'],
  'superThemeTaxonomy': 'http://datos.gob.ar/superThemeTaxonomy.json',
  'issued': '2017-09-28',
  'rights': '2017-09-28',
  'modified': '2017-09-28',
  'spatial': 'ARG',
  'title': 'Datos Programación Macroeconómica',
  'identifier': 'sspm'},
 'dataset': {'publisher': {'mbox': 'datoseconomicos@mecon.gov.ar',
   'name': 'Subsecretaría de Programación Macroeconómica.'},
  'landingPage': 'https://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
  'keyword': ['Información Económica al Día', 'Precios'],
  'superTheme': ['ECON'],
  'title': 'Índice de Precios Internos Básicos al por Mayor (IPIB) hasta cuatro dígitos',
  'language': ['SPA'],
  'issued': '2017-09-

Comprobamos que los metadatos de la API sean iguales a los originales

In [49]:
def compare_metadata(serie_meta, original_meta, equivalent_dict_keys):
    comparisons = []
    for key, value in equivalent_dict_keys.items():
        if isinstance(value, dict):
            for api_key, original_key in value.items():
                print(key, api_key, original_key)
                comparison = serie_meta[key][api_key] == original_meta[original_key]
                comparisons.append('{} {} == {}: {}'.format(key, api_key, original_key, comparison))
    return comparisons
    

In [50]:
keys = {
    'catalog': {
        'identifier': 'catalogo_id',
    },
    'dataset': {
        'identifier': 'dataset_id',
    },
    'distribution': {
        'identifier': 'distribution_id',
    },
    'field': {
        'id': 'serie_id',
    },
}

compare_metadata(serie_meta, orig_meta, keys)

catalog identifier catalogo_id
dataset identifier dataset_id
distribution identifier distribution_id


KeyError: 'distribution_id'

In [ ]:
serie_meta['catalog']['identifier'] == orig_meta['catalogo_id']

In [ ]:
serie_meta['dataset']['identifier'] == str(orig_meta['dataset_id'])

In [ ]:
serie_meta['distribution']['identifier'] == str(orig_meta['distribucion_id'])

In [ ]:
serie_meta['field']['id'] == str(orig_meta['serie_id'])